In [1]:
# importing libraries
import pandas as pd # data science essentials
import numpy as np
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # enhanced data visualization
import statsmodels.formula.api as smf # regression modeling
from sklearn.model_selection import train_test_split # train/test split
from sklearn.linear_model import LinearRegression
import sklearn.linear_model

# setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


# specifying the path and file name
file = './datasets/Apprentice_Chef_Dataset.xlsx'


# reading the file into Python
chef = pd.read_excel(file)


# checking the file
chef.head(n = 5)

,REVENUE,CROSS_SELL_SUCCESS,NAME,EMAIL,FIRST_NAME,FAMILY_NAME,TOTAL_MEALS_ORDERED,UNIQUE_MEALS_PURCH,CONTACTS_W_CUSTOMER_SERVICE,PRODUCT_CATEGORIES_VIEWED,AVG_TIME_PER_SITE_VISIT,MOBILE_NUMBER,CANCELLATIONS_BEFORE_NOON,CANCELLATIONS_AFTER_NOON,TASTES_AND_PREFERENCES,PC_LOGINS,MOBILE_LOGINS,WEEKLY_PLAN,EARLY_DELIVERIES,LATE_DELIVERIES,PACKAGE_LOCKER,REFRIGERATED_LOCKER,AVG_PREP_VID_TIME,LARGEST_ORDER_SIZE,MASTER_CLASSES_ATTENDED,MEDIAN_MEAL_RATING,AVG_CLICKS_PER_VISIT,TOTAL_PHOTOS_VIEWED
0,393.0,1,Saathos,saathos@unitedhealth.com,Saathos,Saathos,14,6,12,10,48.00,1,3,1,1,5,2,0,0,2,0,0,33.4,1,0,1,17,0
1,1365.0,1,Alysanne Osgrey,alysanne.osgrey@ge.org,Alysanne,Osgrey,87,3,8,8,40.35,1,0,0,1,5,1,12,0,2,0,0,84.8,1,0,3,13,170
2,800.0,1,Edwyd Fossoway,edwyd.fossoway@jnj.com,Edwyd,Fossoway,15,7,11,5,19.77,1,3,0,1,6,1,1,0,1,0,0,63.0,1,0,2,16,0
3,600.0,1,Eleyna Westerling,eleyna.westerling@ge.org,Eleyna,Westerling,13,6,11,5,90.00,1,2,0,1,6,1,14,0,3,0,0,43.8,1,0,2,14,0
4,1490.0,1,Elyn Norridge,elyn.norridge@jnj.com,Elyn,Norridge,47,8,6,10,40.38,1,0,0,0,5,1,5,0,8,0,0,84.8,1,1,3,12,205


In [2]:
#checking for the number of rows(observations) and features(columns/variables)
chef.shape

print(f"""
Size of Original Dataset
------------------------
Observations: {chef.shape[0]}
Features:     {chef.shape[1]}
""")


Size of Original Dataset
------------------------
Observations: 1946
Features:     28



In [3]:
# checking for missing values in the dataset
chef.isnull().sum(axis = 0)

REVENUE                         0
CROSS_SELL_SUCCESS              0
NAME                            0
EMAIL                           0
FIRST_NAME                      0
FAMILY_NAME                    47
TOTAL_MEALS_ORDERED             0
UNIQUE_MEALS_PURCH              0
CONTACTS_W_CUSTOMER_SERVICE     0
PRODUCT_CATEGORIES_VIEWED       0
AVG_TIME_PER_SITE_VISIT         0
MOBILE_NUMBER                   0
CANCELLATIONS_BEFORE_NOON       0
CANCELLATIONS_AFTER_NOON        0
TASTES_AND_PREFERENCES          0
PC_LOGINS                       0
MOBILE_LOGINS                   0
WEEKLY_PLAN                     0
EARLY_DELIVERIES                0
LATE_DELIVERIES                 0
PACKAGE_LOCKER                  0
REFRIGERATED_LOCKER             0
AVG_PREP_VID_TIME               0
LARGEST_ORDER_SIZE              0
MASTER_CLASSES_ATTENDED         0
MEDIAN_MEAL_RATING              0
AVG_CLICKS_PER_VISIT            0
TOTAL_PHOTOS_VIEWED             0
dtype: int64

In [4]:
#looping to detect features with missing values
for col in chef:

    # creating columns with 1s if missing and 0 if not
    if chef[col].isnull().astype(int).sum() > 0:
        chef['m_'+col] = chef[col].isnull().astype(int)


#summing the missing value flags to check the results of the loop above
chef[['m_FAMILY_NAME']].sum(axis = 0)

m_FAMILY_NAME    47
dtype: int64

In [5]:
# instantiating an imputation value
fill = "none"

# imputing 'FAMILY_NAME'
chef['FAMILY_NAME'] = chef['FAMILY_NAME'].fillna(fill)

In [6]:
# checking if all missing values have been taken care of
chef.isnull().any(axis = 0).any(axis = 0)

False

In [7]:
#splitting emails

# placeholder list
placeholder_lst = []

# looping over each email address
for index, col in chef.iterrows():
    
    # splitting email domain at '@'
    split_email = chef.loc[index, 'EMAIL'].split(sep = '@')
    
    # appending placeholder_lst with the results
    placeholder_lst.append(split_email)
    

# converting placeholder_lst into a DataFrame 
email_df = pd.DataFrame(placeholder_lst)


# displaying the results
email_df

,0,1
0,saathos,unitedhealth.com
1,alysanne.osgrey,ge.org
2,edwyd.fossoway,jnj.com
3,eleyna.westerling,ge.org
4,elyn.norridge,jnj.com
...,...,...
1941,obara.sand,yahoo.com
1942,quentyn.blackwood,yahoo.com
1943,rhonda.rowan,gmail.com
1944,turnip,yahoo.com


In [8]:
#concatenating with original DataFrame

# safety measure in case of multiple concatenations
chef = pd.read_excel(file)


# renaming column to concatenate
email_df.columns = ['0' , 'email_domain']


# concatenating email_domain with chef DataFrame
chef = pd.concat([chef, email_df['email_domain']],
                     axis = 1)


# printing value counts of email_domain
chef.loc[: ,'email_domain'].value_counts()

gmail.com           303
protonmail.com      284
yahoo.com           274
msn.com              72
aol.com              69
passport.com         64
hotmail.com          63
live.com             62
me.com               59
amex.com             30
jnj.com              28
cocacola.com         28
mcdonalds.com        28
merck.com            28
nike.com             27
apple.com            27
ge.org               26
ibm.com              26
dupont.com           26
chevron.com          25
microsoft.com        25
travelers.com        24
exxon.com            24
unitedhealth.com     24
boeing.com           23
mmm.com              22
verizon.com          22
pg.com               22
caterpillar.com      22
disney.com           21
walmart.com          21
pfizer.com           20
visa.com             20
jpmorgan.com         19
cisco.com            18
goldmansacs.com      18
unitedtech.com       18
homedepot.com        17
intel.com            17
Name: email_domain, dtype: int64

In [9]:
# email domain types
personal_email_domain = ['@gmail.com', '@yahoo.com']
company_email_domain  = ['@protonmail.com','@amex.com','@jnj.com','@mcdonalds.com',
                        '@merck.com','@cocacola.com','@nike.com','@apple.com','@ge.org','@dupont.com','@ibm.com',
                        '@chevron.com','@microsoft.com','@exxon.com','@travelers.com','@unitedhealth.com',
                        '@boeing.com','@mmm.com','@pg.com','@caterpillar.com','@verizon.com','@disney.com',
                        '@walmart.com','@visa.com','@pfizer.com','@jpmorgan.com','@unitedtech.com','@goldmansacs.com',
                        '@cisco.com','@intel.com','@homedepot.com']
junk_email_domain = ['@me.com','@aol.com','@hotmail.com','@live.com','@msn.com','@passport.com']


# placeholder list, creating an empty list
placeholder_lst = []


#looping to group observations by domain type
for domain in chef['email_domain']:
    
    if '@' + domain in personal_email_domain:
        placeholder_lst.append('personal')
        

    elif '@' + domain in company_email_domain:
        placeholder_lst.append('company')
        
    elif '@' + domain in junk_email_domain:
        placeholder_lst.append('junk')

    else:
            print('Unknown')


# concatenating with original DataFrame
chef['domain_group'] = pd.Series(placeholder_lst)


# checking results
chef['domain_group'].value_counts()

company     980
personal    577
junk        389
Name: domain_group, dtype: int64

In [10]:
#creating dummies for the domain_group and putting the data into a new dataframe with the created dummies
chef_data = pd.get_dummies(chef, columns = ['domain_group'])
chef_data.head() #checking if the columns are created using head()

,REVENUE,CROSS_SELL_SUCCESS,NAME,EMAIL,FIRST_NAME,FAMILY_NAME,TOTAL_MEALS_ORDERED,UNIQUE_MEALS_PURCH,CONTACTS_W_CUSTOMER_SERVICE,PRODUCT_CATEGORIES_VIEWED,AVG_TIME_PER_SITE_VISIT,MOBILE_NUMBER,CANCELLATIONS_BEFORE_NOON,CANCELLATIONS_AFTER_NOON,TASTES_AND_PREFERENCES,PC_LOGINS,MOBILE_LOGINS,WEEKLY_PLAN,EARLY_DELIVERIES,LATE_DELIVERIES,PACKAGE_LOCKER,REFRIGERATED_LOCKER,AVG_PREP_VID_TIME,LARGEST_ORDER_SIZE,MASTER_CLASSES_ATTENDED,MEDIAN_MEAL_RATING,AVG_CLICKS_PER_VISIT,TOTAL_PHOTOS_VIEWED,email_domain,domain_group_company,domain_group_junk,domain_group_personal
0,393.0,1,Saathos,saathos@unitedhealth.com,Saathos,Saathos,14,6,12,10,48.00,1,3,1,1,5,2,0,0,2,0,0,33.4,1,0,1,17,0,unitedhealth.com,1,0,0
1,1365.0,1,Alysanne Osgrey,alysanne.osgrey@ge.org,Alysanne,Osgrey,87,3,8,8,40.35,1,0,0,1,5,1,12,0,2,0,0,84.8,1,0,3,13,170,ge.org,1,0,0
2,800.0,1,Edwyd Fossoway,edwyd.fossoway@jnj.com,Edwyd,Fossoway,15,7,11,5,19.77,1,3,0,1,6,1,1,0,1,0,0,63.0,1,0,2,16,0,jnj.com,1,0,0
3,600.0,1,Eleyna Westerling,eleyna.westerling@ge.org,Eleyna,Westerling,13,6,11,5,90.00,1,2,0,1,6,1,14,0,3,0,0,43.8,1,0,2,14,0,ge.org,1,0,0
4,1490.0,1,Elyn Norridge,elyn.norridge@jnj.com,Elyn,Norridge,47,8,6,10,40.38,1,0,0,0,5,1,5,0,8,0,0,84.8,1,1,3,12,205,jnj.com,1,0,0


In [11]:
# counting the number of zeroes
TOTAL_MEALS_ORDERED_zero  = len(chef_data['TOTAL_MEALS_ORDERED'][chef_data['TOTAL_MEALS_ORDERED'] == 0])
TOTAL_PHOTOS_VIEWED_zero  = len(chef_data['TOTAL_PHOTOS_VIEWED'][chef_data['TOTAL_PHOTOS_VIEWED'] == 0])
WEEKLY_PLAN_zero     = len(chef_data['WEEKLY_PLAN'][chef_data['WEEKLY_PLAN'] == 0])

# printing a table of the results of the number of zeroes
print(f"""
                             0\t\tOthers
                           ---------------------
TOTAL_MEALS_ORDERED       | {TOTAL_MEALS_ORDERED_zero}\t\t{len(chef_data) - TOTAL_MEALS_ORDERED_zero}
TOTAL_PHOTOS_VIEWED       | {TOTAL_PHOTOS_VIEWED_zero}\t{len(chef_data) - TOTAL_PHOTOS_VIEWED_zero}
WEEKLY_PLAN               | {WEEKLY_PLAN_zero}\t\t{len(chef_data) - WEEKLY_PLAN_zero}

""")


                             0		Others
                           ---------------------
TOTAL_MEALS_ORDERED       | 0		1946
TOTAL_PHOTOS_VIEWED       | 1140	806
WEEKLY_PLAN               | 468		1478




In [12]:
# dummy variable for photo views and weekly plan(0 or at least 1). 
chef_data['HAS_VIEWED_PHOTO']   = 0
chef_data['HAS_WEEKLY_PLAN'] = 0

# iterating over each original column to change values in the new feature columns
for index, value in chef_data.iterrows():
    
    # TOTAL_PHOTOS_VIEWED
    if chef_data.loc[index, "TOTAL_PHOTOS_VIEWED"] > 0:
        chef_data.loc[index, 'HAS_VIEWED_PHOTO'] = 1


    # WEEKLY_PLAN
    if chef_data.loc[index, "WEEKLY_PLAN"] > 0:
        chef_data.loc[index, 'HAS_WEEKLY_PLAN'] = 1
        

In [13]:
# log transforming of variables and saving them to the dataset
chef_data['log_revenue'] = np.log10(chef_data['REVENUE'])

chef_data['log_TOTAL_MEALS_ORDERED'] = np.log10(chef_data['TOTAL_MEALS_ORDERED'])

chef_data['log_AVG_PREP_VID_TIME'] = np.log10(chef_data['AVG_PREP_VID_TIME'])

chef_data['log_AVG_TIME_PER_SITE_VISIT'] = np.log10(chef_data['AVG_TIME_PER_SITE_VISIT'])

chef_data['log_LARGEST_ORDER_SIZE'] = np.log10(chef_data['LARGEST_ORDER_SIZE'])

chef_data['log_UNIQUE_MEALS_PURCH'] = np.log10(chef_data['UNIQUE_MEALS_PURCH'])

In [14]:
#calculating / creating new variables and adding them to the dataset chef_data
chef_data['late_over_total'] = chef_data.LATE_DELIVERIES / chef_data.TOTAL_MEALS_ORDERED # Late Deliveries per how many meals ordered

chef_data['product_cat_viewed_over_total_logins'] = chef_data.PRODUCT_CATEGORIES_VIEWED / (chef_data.PC_LOGINS + chef_data.MOBILE_LOGINS) # Categories viewed per total number of logins 

chef_data['total_logins_over_total_meals_ordered'] = (chef_data.MOBILE_LOGINS + chef_data.PC_LOGINS) / chef_data.TOTAL_MEALS_ORDERED # Total Logins per total meals ordered

chef_data['total_unique_orders'] =  chef_data.UNIQUE_MEALS_PURCH / chef_data.TOTAL_MEALS_ORDERED # Total Unique orders

chef_data['total_orders_contact_custServ'] = chef_data.CONTACTS_W_CUSTOMER_SERVICE / chef_data.TOTAL_MEALS_ORDERED # Orders contacted the customer service

chef_data['total_orders_without_cancellations'] = chef_data.TOTAL_MEALS_ORDERED - chef_data.CANCELLATIONS_BEFORE_NOON #Orders with no cancellation

chef_data['late_deli_over_total_meals'] = chef_data.LATE_DELIVERIES / chef_data.TOTAL_MEALS_ORDERED # Late delivery of the total orders

In [15]:
# creating a (Pearson) correlation matrix
df_corr = chef_data.corr().round(2)


# printing (Pearson) correlations with log_revenue
print(df_corr.loc['log_revenue'].sort_values(ascending = False))

log_revenue                              1.00
REVENUE                                  0.95
log_TOTAL_MEALS_ORDERED                  0.69
log_AVG_PREP_VID_TIME                    0.67
AVG_PREP_VID_TIME                        0.65
MEDIAN_MEAL_RATING                       0.65
TOTAL_MEALS_ORDERED                      0.61
total_orders_without_cancellations       0.61
MASTER_CLASSES_ATTENDED                  0.47
log_LARGEST_ORDER_SIZE                   0.45
LARGEST_ORDER_SIZE                       0.45
TOTAL_PHOTOS_VIEWED                      0.43
HAS_VIEWED_PHOTO                         0.39
log_AVG_TIME_PER_SITE_VISIT              0.15
AVG_TIME_PER_SITE_VISIT                  0.14
MOBILE_NUMBER                            0.04
product_cat_viewed_over_total_logins     0.04
PRODUCT_CATEGORIES_VIEWED                0.04
domain_group_personal                    0.01
WEEKLY_PLAN                              0.01
CANCELLATIONS_BEFORE_NOON                0.01
TASTES_AND_PREFERENCES            

In [16]:
#dropping variables to proceed with regression. 
#The dropped variables include variables log transformed so that we have only the log of those variables in dataset.
#also dropping variables that have been encoded
chef_df = chef_data.drop(['REVENUE', 'log_revenue',
                        'NAME','EMAIL','AVG_PREP_VID_TIME','AVG_TIME_PER_SITE_VISIT',
                        'FIRST_NAME','TOTAL_MEALS_ORDERED','LARGEST_ORDER_SIZE','UNIQUE_MEALS_PURCH',
                        'FAMILY_NAME','email_domain'],
                         axis = 1)


# preparing response variables
chef_target = chef_data.loc[ : , 'REVENUE']
log_chef_target = chef_data.loc[ : , 'log_revenue']

# preparing training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
            chef_df,
            log_chef_target,
            test_size = 0.25,
            random_state = 219)


# checking the shapes of the datasets
print(f"""
Training Data
-------------
X-side: {x_train.shape}
y-side: {y_train.shape}

Testing Data
------------
X-side: {x_test.shape}
y-side: {y_test.shape}
""")


Training Data
-------------
X-side: (1459, 35)
y-side: (1459,)

Testing Data
------------
X-side: (487, 35)
y-side: (487,)



In [17]:
# making a copy of original
chef_data_df = chef_data.copy()



# dropping REVENUE and other unneccesary explanatory variable set
chef_data_df = chef_data_df.drop(['REVENUE', 'log_revenue',
                        'NAME','EMAIL','AVG_PREP_VID_TIME','AVG_TIME_PER_SITE_VISIT',
                        'FIRST_NAME','TOTAL_MEALS_ORDERED','LARGEST_ORDER_SIZE','UNIQUE_MEALS_PURCH',
                        'FAMILY_NAME','email_domain'], axis = 1)


# formatting each explanatory variable
for val in chef_data_df:
    print(val, '+')

CROSS_SELL_SUCCESS +
CONTACTS_W_CUSTOMER_SERVICE +
PRODUCT_CATEGORIES_VIEWED +
MOBILE_NUMBER +
CANCELLATIONS_BEFORE_NOON +
CANCELLATIONS_AFTER_NOON +
TASTES_AND_PREFERENCES +
PC_LOGINS +
MOBILE_LOGINS +
WEEKLY_PLAN +
EARLY_DELIVERIES +
LATE_DELIVERIES +
PACKAGE_LOCKER +
REFRIGERATED_LOCKER +
MASTER_CLASSES_ATTENDED +
MEDIAN_MEAL_RATING +
AVG_CLICKS_PER_VISIT +
TOTAL_PHOTOS_VIEWED +
domain_group_company +
domain_group_junk +
domain_group_personal +
HAS_VIEWED_PHOTO +
HAS_WEEKLY_PLAN +
log_TOTAL_MEALS_ORDERED +
log_AVG_PREP_VID_TIME +
log_AVG_TIME_PER_SITE_VISIT +
log_LARGEST_ORDER_SIZE +
log_UNIQUE_MEALS_PURCH +
late_over_total +
product_cat_viewed_over_total_logins +
total_logins_over_total_meals_ordered +
total_unique_orders +
total_orders_contact_custServ +
total_orders_without_cancellations +
late_deli_over_total_meals +


In [18]:
# merging X_train and y_train so that they can be used
chef_data_train = pd.concat([x_train, y_train], axis = 1)


#building a model
lm_best = smf.ols(formula =  """log_revenue ~ CROSS_SELL_SUCCESS +
                                CONTACTS_W_CUSTOMER_SERVICE +
                                MASTER_CLASSES_ATTENDED +
                                MEDIAN_MEAL_RATING +
                                AVG_CLICKS_PER_VISIT +
                                TOTAL_PHOTOS_VIEWED +
                                domain_group_company +
                                domain_group_junk +
                                domain_group_personal +
                                log_TOTAL_MEALS_ORDERED +
                                log_AVG_PREP_VID_TIME +
                                log_LARGEST_ORDER_SIZE +
                                log_UNIQUE_MEALS_PURCH +
                                total_unique_orders +
                                total_orders_contact_custServ 
                                """,
                                data = chef_data_train)


#fitting the model based on the data
results = lm_best.fit()



#printing the summary output for analysis
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            log_revenue   R-squared:                       0.748
Model:                            OLS   Adj. R-squared:                  0.745
Method:                 Least Squares   F-statistic:                     306.0
Date:                Sun, 24 Jan 2021   Prob (F-statistic):               0.00
Time:                        12:08:33   Log-Likelihood:                 1216.8
No. Observations:                1459   AIC:                            -2404.
Df Residuals:                    1444   BIC:                            -2324.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

In [19]:
# declaring set of x-variables
x_variables = ['CROSS_SELL_SUCCESS','CONTACTS_W_CUSTOMER_SERVICE',
                                'MASTER_CLASSES_ATTENDED',
                                'MEDIAN_MEAL_RATING',
                                'AVG_CLICKS_PER_VISIT',
                                'TOTAL_PHOTOS_VIEWED',
                                'domain_group_company',
                                'domain_group_junk',
                                'domain_group_personal',
                                'log_TOTAL_MEALS_ORDERED',
                                'log_AVG_PREP_VID_TIME',
                                'log_LARGEST_ORDER_SIZE',
                                'log_UNIQUE_MEALS_PURCH',
                                'total_unique_orders',
                                'total_orders_contact_custServ']


# looping to make x-variables suitable for models
for val in x_variables:
    print(f"{val} +")

CROSS_SELL_SUCCESS +
CONTACTS_W_CUSTOMER_SERVICE +
MASTER_CLASSES_ATTENDED +
MEDIAN_MEAL_RATING +
AVG_CLICKS_PER_VISIT +
TOTAL_PHOTOS_VIEWED +
domain_group_company +
domain_group_junk +
domain_group_personal +
log_TOTAL_MEALS_ORDERED +
log_AVG_PREP_VID_TIME +
log_LARGEST_ORDER_SIZE +
log_UNIQUE_MEALS_PURCH +
total_unique_orders +
total_orders_contact_custServ +


In [20]:
# applying model in scikit-learn

# Preparing a DataFrame based the analysis above
ols_data   = chef_data.loc[ : , x_variables]


# Preparing the target variable
log_chef_target = chef_data.loc[ : , 'log_revenue']


###############################################
## setting up more than one train-test split ##
###############################################
#splitting the dataset by keeping the test size to 0.25

# FULL x-dataset (normal Y)
x_train_FULL, x_test_FULL, y_train_FULL, y_test_FULL = train_test_split(
            chef_df,     # x-variables
            log_chef_target,   # y-variable
            test_size = 0.25,
            random_state = 219)


# OLS p-value x-dataset (normal Y)
x_train_OLS, x_test_OLS, y_train_OLS, y_test_OLS = train_test_split(
            ols_data,         # x-variables
            log_chef_target,   # y-variable
            test_size = 0.25,
            random_state = 219)


In [21]:
# INSTANTIATING OLS model object
lr = LinearRegression()


# FITTING to the training data
lr_fit = lr.fit(x_train_OLS, y_train_OLS)


# PREDICTING on new data
lr_pred = lr_fit.predict(x_test_OLS)


# SCORING the results
print('OLS Training Score :', lr.score(x_train_OLS, y_train_OLS).round(4))
print('OLS Testing Score  :', lr.score(x_test_OLS, y_test_OLS).round(4))


# saving scoring data for future use
lr_train_score = lr.score(x_train_OLS, y_train_OLS).round(4) # using R-square
lr_test_score  = lr.score(x_test_OLS, y_test_OLS).round(4)   # using R-square


# displaying and saving the gap between training and testing
print('OLS Train-Test Gap :', abs(lr_train_score - lr_test_score).round(4))
lr_test_gap = abs(lr_train_score - lr_test_score).round(4)

OLS Training Score : 0.7479
OLS Testing Score  : 0.75
OLS Train-Test Gap : 0.0021


In [22]:
# zipping each feature name to its coefficient
lr_model_values = zip(chef_df[x_variables].columns,
                      lr_fit.coef_.round(decimals = 2))


# setting up a placeholder list to store model features
lr_model_lst = [('intercept', lr_fit.intercept_.round(decimals = 2))]


# printing out each feature-coefficient pair one by one
for val in lr_model_values:
    lr_model_lst.append(val)
    

# checking the results
for pair in lr_model_lst:
    print(pair)

('intercept', 1.98)
('CROSS_SELL_SUCCESS', -0.02)
('CONTACTS_W_CUSTOMER_SERVICE', 0.03)
('MASTER_CLASSES_ATTENDED', 0.03)
('MEDIAN_MEAL_RATING', 0.05)
('AVG_CLICKS_PER_VISIT', -0.0)
('TOTAL_PHOTOS_VIEWED', 0.0)
('domain_group_company', -0.0)
('domain_group_junk', -0.01)
('domain_group_personal', 0.01)
('log_TOTAL_MEALS_ORDERED', 0.12)
('log_AVG_PREP_VID_TIME', 0.48)
('log_LARGEST_ORDER_SIZE', -0.1)
('log_UNIQUE_MEALS_PURCH', -0.21)
('total_unique_orders', 0.21)
('total_orders_contact_custServ', -0.46)


In [23]:
# INSTANTIATING Bayesian Automatic Relevance Determination (ARD) model object
ard_model = sklearn.linear_model.ARDRegression(normalize  = False)


# FITTING the training data
ard_fit = ard_model.fit(x_train_FULL, y_train_FULL)


# PREDICTING on new data
ard_pred = ard_fit.predict(x_test_FULL)


print('Training Score:', ard_model.score(x_train_FULL, y_train_FULL).round(4))
print('Testing Score :',  ard_model.score(x_test_FULL, y_test_FULL).round(4))


# saving scoring data for future use
ard_train_score = ard_model.score(x_train_FULL, y_train_FULL).round(4)
ard_test_score  = ard_model.score(x_test_FULL, y_test_FULL).round(4)


# displaying and saving the gap between training and testing
print('ARD Train-Test Gap :', abs(ard_train_score - ard_test_score).round(4))
ard_test_gap = abs(ard_train_score - ard_test_score).round(4)

Training Score: 0.7534
Testing Score : 0.7513
ARD Train-Test Gap : 0.0021


In [24]:
# zipping each feature name to its coefficient
ard_model_values = zip(chef_df.columns, ard_fit.coef_.round(decimals = 5))


# setting up a placeholder list to store model features
ard_model_lst = [('intercept', ard_fit.intercept_.round(decimals = 2))]


# printing out each feature-coefficient pair one by one
for val in ard_model_values:
    ard_model_lst.append(val)
    

# printing the results using loop
for pair in ard_model_lst:
    print(pair)

('intercept', 1.7)
('CROSS_SELL_SUCCESS', -0.013)
('CONTACTS_W_CUSTOMER_SERVICE', 0.03266)
('PRODUCT_CATEGORIES_VIEWED', 0.0)
('MOBILE_NUMBER', 0.0)
('CANCELLATIONS_BEFORE_NOON', 0.0)
('CANCELLATIONS_AFTER_NOON', 0.0)
('TASTES_AND_PREFERENCES', 0.00933)
('PC_LOGINS', 0.0)
('MOBILE_LOGINS', -0.01451)
('WEEKLY_PLAN', 0.0)
('EARLY_DELIVERIES', 0.0)
('LATE_DELIVERIES', 0.0)
('PACKAGE_LOCKER', -0.01151)
('REFRIGERATED_LOCKER', 0.0)
('MASTER_CLASSES_ATTENDED', 0.02534)
('MEDIAN_MEAL_RATING', 0.05736)
('AVG_CLICKS_PER_VISIT', 0.0)
('TOTAL_PHOTOS_VIEWED', 0.0)
('domain_group_company', 0.0)
('domain_group_junk', 0.0)
('domain_group_personal', 0.01203)
('HAS_VIEWED_PHOTO', 0.02535)
('HAS_WEEKLY_PLAN', -0.01379)
('log_TOTAL_MEALS_ORDERED', 0.1769)
('log_AVG_PREP_VID_TIME', 0.50778)
('log_AVG_TIME_PER_SITE_VISIT', 0.0)
('log_LARGEST_ORDER_SIZE', -0.10792)
('log_UNIQUE_MEALS_PURCH', -0.20742)
('late_over_total', -0.02652)
('product_cat_viewed_over_total_logins', 0.01169)
('total_logins_over_total_m

In [25]:
# dropping coefficients that are equal to zero

# printing out each feature-coefficient pair one by one
for feature, coefficient in ard_model_lst:
        
        if coefficient == 0:
            ard_model_lst.remove((feature, coefficient))

            
# checking the results, should be run multiple times in order to drop the features with 0 coefficient.
for pair in ard_model_lst:
    print(pair)

('intercept', 1.7)
('CROSS_SELL_SUCCESS', -0.013)
('CONTACTS_W_CUSTOMER_SERVICE', 0.03266)
('MOBILE_NUMBER', 0.0)
('CANCELLATIONS_AFTER_NOON', 0.0)
('TASTES_AND_PREFERENCES', 0.00933)
('MOBILE_LOGINS', -0.01451)
('EARLY_DELIVERIES', 0.0)
('PACKAGE_LOCKER', -0.01151)
('MASTER_CLASSES_ATTENDED', 0.02534)
('MEDIAN_MEAL_RATING', 0.05736)
('TOTAL_PHOTOS_VIEWED', 0.0)
('domain_group_junk', 0.0)
('domain_group_personal', 0.01203)
('HAS_VIEWED_PHOTO', 0.02535)
('HAS_WEEKLY_PLAN', -0.01379)
('log_TOTAL_MEALS_ORDERED', 0.1769)
('log_AVG_PREP_VID_TIME', 0.50778)
('log_LARGEST_ORDER_SIZE', -0.10792)
('log_UNIQUE_MEALS_PURCH', -0.20742)
('late_over_total', -0.02652)
('product_cat_viewed_over_total_logins', 0.01169)
('total_logins_over_total_meals_ordered', 0.45866)
('total_unique_orders', 0.18505)
('total_orders_contact_custServ', -0.65031)
('late_deli_over_total_meals', -0.02652)


In [26]:
#declaring the best result by comparing both the models run and printing the score results.

print("ARD is the final model as it is slightly better than OLS with test score of 0.7513, train score of 0.7534 and the\ntrain-test gap of 0.0021\n")

print("Table showing scores of different models run:")
print(f"""
Model      Train Score      Test Score    Train-Test Gap      Model Size
-----      -----------      ----------    -------------      -------------
OLS        {lr_train_score}            {lr_test_score}          {lr_test_gap}             {len(lr_model_lst)}
ARD        {ard_train_score}            {ard_test_score}        {ard_test_gap}             {len(ard_model_lst)}

-------------------------------------------------------------
FINAL MODEL: Bayesian Automatic Relevance Determination (ARD)
-------------------------------------------------------------
""")

print(f"""
Model Coefficients and Variables 
----------------------------------
 {'OLS'}
--------
    {lr_model_lst}

 {'ARD'} 
--------
    {ard_model_lst}

""")


ARD is the final model as it is slightly better than OLS with test score of 0.7513, train score of 0.7534 and the
train-test gap of 0.0021

Table showing scores of different models run:

Model      Train Score      Test Score    Train-Test Gap      Model Size
-----      -----------      ----------    -------------      -------------
OLS        0.7479            0.75          0.0021             16
ARD        0.7534            0.7513        0.0021             26

-------------------------------------------------------------
FINAL MODEL: Bayesian Automatic Relevance Determination (ARD)
-------------------------------------------------------------


Model Coefficients and Variables 
----------------------------------
 OLS
--------
    [('intercept', 1.98), ('CROSS_SELL_SUCCESS', -0.02), ('CONTACTS_W_CUSTOMER_SERVICE', 0.03), ('MASTER_CLASSES_ATTENDED', 0.03), ('MEDIAN_MEAL_RATING', 0.05), ('AVG_CLICKS_PER_VISIT', -0.0), ('TOTAL_PHOTOS_VIEWED', 0.0), ('domain_group_company', -0.0), ('domain

In [27]:
chef_data.to_excel('Apprentice_Chef_regression.xlsx',
                 index = False)